
# University of Calgary Solar Car Team
## Battery Capacity Calculator

### Packages

#### pint 
Python library designed for managing physical quantities and units.

### Car Attributes Description


#### Mass

Total mass of the car (Kg)
- [Helios Total Mass Calculations](https://uofc.sharepoint.com/:x:/r/sites/CalgarySolarCarTeam/Shared%20Documents/0.2%20Mechanical/Helios%20Battery%20Pack/Weight%20Estimates.xlsx?d=w8a2ccea6332b40eeab0cacf0554fe7bf&csf=1&web=1&e=ommbDp)
- Elysia: 600 kg
- Helios: 1000 kg 




#### Frontal Area

Frontal area of the car that's exposed to the oncoming air. (m^2)
- Elysia: 1.70 m^2
- Helios: 1.66 m^2

![Helios Frontal Area](helios_frontal_area.png) ![Elysia Frontal Area](elysia_frontal_area.png)


#### Drag Coefficient 
Dimensionless coefficient that represents the aerodynamic efficiency of the car's shape. Found using CFD in ANSYS
- Elysia: 0.14
- Helios: 0.135

#### Tire Pressure
Tire inflation pressure in psi
- Elysia: 35 psi
- Helios: 35 psi


### Environment Description

#### Air Density 
Density of air at specified temperature and pressure. (kg/m^3)
- 1.164 kg/m^3 at 30 degrees celsius

    ![Air Density](air_density.png)

### Calculation Explanations

#### Rolling Resistance 
Rolling resistance is the force that slows down a car as its tires roll on the road.

$F_r = c_r * N$

where: 
- $F_r$ - Rolling resistance force (N)
- $c_r$ - Rolling coefficient
- N - Weight of the car acting on the tires (N)

Coefficient of rolling resistance $c_r$ can be calculated using the following equation:

Assumption: Air filled tires on dry roads.


$ c_r = 0.005 + (\dfrac{1}{p}) * (0.01 + 0.0095 * (\dfrac{v}{100})^2) $

where: 
- p - Tire pressure (bar)
- v - Velocity (km/h)


#### Drag Force 
Drag force on a car is the resistance it encounters when moving through the air.

$ F_d = \dfrac{1}{2} \rho A c_d v^2 $

where: 

- $F_d$ - Drag force (N)
- $\rho$ - Air density (kg / $m^3$)
- A - Frontal area ($m^2$)
- $c_d$ - Drag coefficient 
- $v$ - Velocity of the car relative to air (m/s)

#### Total Mechanical Force 

#### Mechanical Power

### Code

In [119]:
!pip install pint
import pint
ureg = pint.UnitRegistry()

In [120]:
import pandas as pd

columns = ['scenario', 'mass (kg)', 'frontal_area (m^2)', 'drag_coefficient', 'tire_pressure (psi)', 'velocity (km/h)']
input_df = pd.DataFrame(columns=columns)
input_df.index.name = 'Scenario_ID'
input_df.loc[0] = ['Elysia', 600, 1.7, 0.14, 35, 60]
input_df.loc[1] = ['Helios', 1000, 1.66, 0.135, 35, 60]

input_df

,scenario,mass (kg),frontal_area (m^2),drag_coefficient,tire_pressure (psi),velocity (km/h)
Scenario_ID,,,,,,
0,Elysia,600,1.70,0.140,35,60
1,Helios,1000,1.66,0.135,35,60


In [121]:
number_of_rows = input_df.shape[0]

In [122]:
# Car Variables 

id = 0

scenario = input_df.iloc[id]['scenario']

mass = input_df.iloc[id]['mass (kg)'] * ureg.kilogram
frontal_area = input_df.iloc[id]['frontal_area (m^2)'] * ureg.meter**2
drag_coefficient = input_df.iloc[id]['drag_coefficient']
tire_pressure = input_df.iloc[id]['tire_pressure (psi)'] * ureg.psi
velocity = input_df.iloc[id]['velocity (km/h)'] * ureg.kilometer / ureg.hour


In [123]:
# Environment Variables

air_density = 1.164 * ureg.kilogram / ureg.meter**3
gravity = 9.81 * ureg.meter / ureg.second**2

In [124]:
# Force Calculations 
# DO NOT MODIFY THIS SECTION

drag_force = (0.5 * air_density * frontal_area * drag_coefficient * velocity.to(ureg.meter / ureg.second)**2).to(ureg.newton)

rolling_resistance_coef = 0.005 + (1 * ureg.bar  / tire_pressure.to(ureg.bar)) * (0.01 + 0.0095*pow(velocity/100, 2) * (1 * ureg.hour**2 / ureg.kilometer**2)) 
rolling_resistance_force = (rolling_resistance_coef * mass * gravity).to(ureg.newton)

In [125]:
# Force Output Data
columns = ['scenario', 'rolling_resistance_coef', 'rolling_resistance_force (N)', 'drag_force (N)']
force_output_df = pd.DataFrame(columns=columns)
force_output_df.index.name = 'Scenario_ID'
force_output_df.loc[id] = [scenario, round(rolling_resistance_coef,4), round(rolling_resistance_force,4), round(drag_force,4)]
force_output_df



/Users/billysidharta/anaconda3/envs/personal/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


,scenario,rolling_resistance_coef,rolling_resistance_force (N),drag_force (N)
Scenario_ID,,,,
0,Elysia,0.0106 dimensionless,62.163 newton,38.4767 newton


In [126]:
print(rolling_resistance_force)
print(drag_force)

62.162995165250706 newton
38.476666666666674 newton


In [127]:
output_df.dtypes

scenario                        object
rolling_resistance_coef         object
rolling_resistance_force (N)    object
drag_force (N)                  object
dtype: object